In [188]:
import numpy as np
import math
import csv

# Classification tree

In [166]:
#Load our data
def loadData(name):
    data = []
    with open(name) as openfile:
        readfile = csv.reader(openfile, delimiter=',')
        for line in readfile:
            if len(line) > 1:
                data.append(line)
    return(data)

In [510]:
class classificationTree():
    def __init__(self, n_min=5): #Create the tree with leaf size of n_min % of total data.
        self.n_min = n_min
        self.root = {}
    
    #Information gain of splitting on data[axis] = name
    def __infoGainDiscreet(self, data, axis, name):
        #Place data into respective groups based on name.
        group1 = []
        group2 = []
        for entry in data:
            if entry[axis] == name:
                group1.append(entry)
            else:
                group2.append(entry)
        group1 = np.array(group1) #Value in range
        group2 = np.array(group2) #Value out of range
        group1_entropy = self.__entropy(group1) #Calculate the entropies
        group2_entropy = self.__entropy(group2) #Calculate the entropies
        resultEntropy = (len(group1)/len(data))*group1_entropy + (len(group2)/len(data))*group2_entropy #Ave entropy
        return(self.__entropy(data) - resultEntropy)
    
    #Information gain of splitting on value > min_val
    def __infoGainContinuous(self, data, axis, min_val = None):
        group1 = []
        group2 = []
        for entry in data:
            if float(entry[axis]) > min_val:
                group1.append(entry)
            else:
                group2.append(entry)
        group1 = np.array(group1) #Value in range
        group2 = np.array(group2) #Value out of range
        group1_entropy = self.__entropy(group1) #Calculate the entropies
        group2_entropy = self.__entropy(group2) #Calculate the entropies
        resultEntropy = (len(group1)/len(data))*group1_entropy + (len(group2)/len(data))*group2_entropy #Ave entropy
        return(self.__entropy(data) - resultEntropy)
    
    #Entropy of a group with respect to our targets (self.labels)
    def __entropy(self, group):
        totalcount = len(group) #Total number of inputs
        entropy = 0 #Total entropy.
        #Calculated the weighted entropy of each label.
        for label in self.labels:
            true = np.array([x for x in group if label in x])
            false = np.array([x for x in group if label not in x])
            if len(true) != 0: #Log of 0.0 is undefined.
                entropy -= (len(true)/totalcount) * math.log((len(true)/totalcount), 2)
        return(entropy)
        
    
    #Finds the feature to split with the highest information gain.
    #For discreet data, we use the median as the splitting category.
    #For continuous data, we use the average
    #Returns the column(feature), value
    def __find_best_feature(self, data):
        highest_info_gain = 0.0 #This tracks the highest we found
        highest_column = 0 #This tracks the feature we should split by.
        splitby = "" #This tracks the name (discreet) or min_split (continuous)
        
        for i in range(len(data[0])-1): #For each feature (column) minus label.
            #Determine the type of feature.
            featuretype = ""
            if 'int' in str(type(data[0][i])) or 'float' in str(type(data[0][i])):
                featuretype = "continuous"
            elif 'str' in str(type(data[0][i])):
                featuretype = "discreet"
            
            #If our feature is discreet, we check info gain on each name(type)
            if featuretype == "discreet":
                #Get only unique categories.
                possibility = set()
                for j in range(len(data)):
                    possibility.add(data[j][i])
                #Now iterate over unique names
                for name in possibility:
                    info_gain = self.__infoGainDiscreet(data, i, name)
                    if(info_gain > highest_info_gain): #If we exceeded existin info_gain
                        highest_info_gain = info_gain #Update highest values.
                        highest_column = i
                        splitby = name

            #If the feature is continuous, search over the whole range as specified in docs.
            #"For determining the optimal threshold for splitting you will need to search over
            #all possible thresholds for a given feature". This is slow.
            elif featuretype == "continuous":
                for j in range(len(data)):
                    info_gain = self.__infoGainContinuous(data, i, data[j][i])
                    if(info_gain > highest_info_gain): #If we exceeded existin info_gain
                        highest_info_gain = info_gain #Update highest values.
                        highest_column = i
                        splitby = str(data[j][i])

        return(highest_column, splitby)
    
#     def splitNode(self, data, column, splitby):
#         return()
                
    #Fit the data into our tree.
    #Continuous data should be numerical. Categorical should be string.
    def fit(self, data, label):
        #Attach the labels to the end of our data.
        for i in range(len(data)):
            data[i].append(label[i])
        self.labels = list(set(label))   #These are the unique labels.
        self.size = len(data) #size of our dataset
        self.n_min = (self.n_min * self.size)/100 #Minimum leaf size.
        
        #Use ID3 algorithm. 
        #1. Create a root node.
#         self.root = {
#             "data":data,
#             "column":None,
#             "splitby":None,
#             "left":None,
#             "right":None}
        #calculate entropy 
        print(self.__infoGainDiscreet(data, 3, "40"))
        print(self.__infoGainContinuous(data, 3, 1.0))
        column, splitby = self.__find_best_feature(data)
        print(column, splitby)

## Classification on the Iris dataset

In [511]:
#Load the iris dataset.
data = loadData("iris.csv")
label = [x[-1] for x in data]
data = [x[:-1] for x in data]
#Set the correct data types.
for i in range(len(data)):
    data[i][0] = float(data[i][0])
    data[i][1] = float(data[i][1])
    data[i][2] = float(data[i][2])
    data[i][3] = float(data[i][3])


In [512]:
tree = classificationTree(n_min = 5) #n_min = minimum percentage of data as leafnode size.
tree.fit(data,label)

0.0
0.7632957516786808
2 1.9
